# RFT-Lab — Answer Decoder

This notebook implements the **Answer Decoder**.

Important rule:
- Generation happens ONLY after reasoning is complete

Pipeline so far:
Input → Understanding Encoder → Reasoning Block → Answer Decoder

This decoder:
- Converts reasoned latent state into text
- Does NOT reason
- Does NOT modify reasoning output
- Produces short, controlled answers

This module is frozen after validation.


## Step 0: Imports

We use PyTorch for decoding
and a pretrained tokenizer for text conversion.


In [2]:
import torch
import torch.nn as nn
from transformers import GPT2TokenizerFast

## Step 1: Tokenizer

The decoder must convert latent outputs into text.

We use a fixed GPT-2 tokenizer.
This ensures consistency between training and inference.


In [4]:
# Load GPT-2 tokenizer for fast text tokenization
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# Set padding token to end-of-sequence token (GPT-2 has no native pad token)
tokenizer.pad_token = tokenizer.eos_token # blank space is padding ,we use EOS token as that blank space

## Step 2: Decoder Head

This layer maps latent vectors
to vocabulary logits.

Important:
- This layer does NOT reason
- It only translates representations into words


In [6]:
class AnswerDecoderHead(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()

        # Converts internal representations into vocabulary scores
        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # Input shape: (batch, seq_len, d_model)
        # Output shape: (batch, seq_len, vocab_size)
        logits = self.output_layer(x)
        return logits
# This decoder head is the mouth of the model.


## Step 3: Controlled Decoding Strategy

We want:
- Short answers
- Stable outputs
- No hallucinated long text

So we use:
- Argmax decoding (deterministic)
- No temperature tricks


In [7]:
def decode_tokens(logits):
    # Pick the most probable token at each position
    token_ids = torch.argmax(logits, dim=-1)
    return token_ids
# We keep decoding simple and deterministic for safety

## Step 4: Answer Decoder

This module:
- Accepts reasoned latent state
- Generates final text output
- Does not modify reasoning

This is the LAST step in the pipeline.


In [9]:
class AnswerDecoder(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()

        # Decoder head converts latent state to logits
        self.decoder_head = AnswerDecoderHead(d_model, vocab_size)

    def forward(self, reasoned_state):
        # Convert latent representation to logits
        logits = self.decoder_head(reasoned_state)

        # Convert logits to token IDs
        token_ids = decode_tokens(logits)

        return token_ids
    # This AnswerDecoder is the speaking part of the model.


## Step 5: End-to-End Test

We simulate the output of the Reasoning Block
and decode it into text.


In [10]:
# Simulated reasoned output from Reasoning Block
reasoned_state = torch.randn(1, 6, 128)

# Initialize decoder
decoder = AnswerDecoder(
    d_model=128,
    vocab_size=tokenizer.vocab_size
)

# Decode tokens
token_ids = decoder(reasoned_state)

# Convert tokens to text
decoded_text = tokenizer.batch_decode(
    token_ids,
    skip_special_tokens=True
)

decoded_text


[' Prometheuswives Cust Feet Tree ']

## Output Explanation

The decoder returns:
- token_ids → predicted vocabulary tokens
- decoded_text → final user-visible answer

Key property:
Reasoning has already happened.
The decoder only verbalizes it.

This guarantees:
- No hidden reasoning during generation
- Clear separation of responsibilities


# **This notebook demonstrates:**
- Strict separation of reasoning and generation
- Safe, deterministic decoding
- Production-ready design
- Explainable AI pipeline